In [1]:
!pip install gdown
!pip install -q pyvi

import pandas as pd
import ast
from pyvi.ViTokenizer import tokenize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 59.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.4 MB/s eta 0:00:00


In [2]:
corpus = pd.read_csv("/kaggle/input/pvi-token/corpus_pvi.csv") #corpus duoc word segmentation bang pvi
final_org = pd.read_csv("/kaggle/input/split-data/train.csv")
len(final_org)

109456

In [3]:
def process_context(context):
    return [
        item.strip('[]\'\"').replace('\\n', ' ').strip('""')
        for item in context.split('\n')
        if item.strip()
    ]

def process_cid(cid_str):
    return [int(x) for x in cid_str.strip('[]').split()]

# Áp dụng các hàm vào DataFrame
final_org['cid'] = final_org['cid'].apply(process_cid)
final_org['context'] = final_org['context'].apply(process_context)


In [4]:
for idx, row in final_org.iterrows():
    for i, (cid, _) in enumerate(zip(row['cid'], row['context'])):
        if cid in corpus['cid'].values: 
            final_org.at[idx, 'context'][i] = corpus.loc[corpus['cid'] == cid, 'tokens'].values[0]


for idx, row in final_org.iterrows():
    for i, (cid, context) in enumerate(zip(row['cid'], row['context'])):
        if cid not in corpus['cid'].values: 
            final_org.at[idx, 'context'][i] = tokenize(context)

final_org["question"] = final_org["question"].apply(lambda x: tokenize(x))

In [5]:
final_org.head()

,question,context,cid,qid
0,Người học ngành quản_lý khai_thác công_trình t...,"[Khả_năng học_tập , nâng cao_trình_độ - Khối_l...",[62492],161615
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,[Nội_dung lồng_ghép vấn_đề bình_đẳng giới tron...,[151154],80037
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"[Điều 20 . Ưu_đãi về thời_gian miễn thuế , giả...",[75071],124074
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"[Điều_kiện được hưởngCán bộ quản_lý , giáo_viê...",[225897],146841
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"[Nguyên_tắc áp dụng1 . Trường_hợp công_chức , ...",[68365],6176


In [20]:
data = {
    "query": [],
    "answer": [],
    "cid": [],
    "qid": []
}

for idx, row in final_org.iterrows():
    query = row['question']
    qid = row['qid']
    for pos, cid in zip(row['context'], row['cid']):
        if len(pos) <= 20: 
            continue
        data['query'].append(query)
        data['qid'].append(qid)
        data['answer'].append(pos)
        data['cid'].append(cid)

In [22]:
# # Duyệt qua dữ liệu trong data và in ra từng dòng
# for i in range(len(data['query'])):
#     query = data['query'][i]
#     answer = data['answer'][i]
#     cid = data['cid'][i]
#     qid = data['qid'][i]
#     if len(answer) <= 30:
#         # In từng phần tử
#         print(len(answer))
#         print(f"Query: {query}")
#         print(f"Answer: {answer}")
#         print(f"CID: {cid}")
#         print(f"QID: {qid}")
#         print("-" * 50)  # Để phân cách các dòng
# # Kiểm tra số lượng sample trong data
num_samples = len(data['query'])  # Hoặc bạn có thể dùng data['answer'], data['cid'], data['qid']
print(f"Tổng số sample là: {num_samples}")


Tổng số sample là: 122327


In [23]:
df = pd.DataFrame(data)
df.to_csv("train_bi_3.csv", index=False, encoding="utf-8")

In [24]:
data_csv = pd.read_csv("/kaggle/working/train_bi_3.csv")
data_csv.head()

,query,answer,cid,qid
0,Người học ngành quản_lý khai_thác công_trình t...,"Khả_năng học_tập , nâng cao_trình_độ - Khối_lư...",62492,161615
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,151154,80037
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"Điều 20 . Ưu_đãi về thời_gian miễn thuế , giảm...",75071,124074
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"Điều_kiện được hưởngCán bộ quản_lý , giáo_viên...",225897,146841
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"Nguyên_tắc áp dụng1 . Trường_hợp công_chức , v...",68365,6176


In [25]:
data_csv['answer'] = data_csv['answer'].str.replace(r'(\D)(\d)', r'\1 \2', regex=True)
data_csv['answer'] = data_csv['answer'].str.replace(r'(\d+)(\D)', r'\1 \2', regex=True)  # Số + Chữ
data_csv.head()

,query,answer,cid,qid
0,Người học ngành quản_lý khai_thác công_trình t...,"Khả_năng học_tập , nâng cao_trình_độ - Khối_lư...",62492,161615
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,151154,80037
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"Điều 20 . Ưu_đãi về thời_gian miễn thuế , gi...",75071,124074
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"Điều_kiện được hưởngCán bộ quản_lý , giáo_viên...",225897,146841
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"Nguyên_tắc áp dụng 1 . Trường_hợp công_chức ,...",68365,6176


In [26]:
data_csv.to_csv("train_bi_2.1.csv", index=False, encoding="utf-8")

In [55]:
import zipfile

csv_file = "/kaggle/working/train_bi_2.csv"  
zip_path = "train_bi_2.zip" 

with zipfile.ZipFile(zip_path, 'w') as zipf:
    zipf.write(csv_file)
    print(f"Đã nén {csv_file} vào {zip_path}")

Đã nén /kaggle/working/train_bi_2.csv vào train_bi_2.zip


In [54]:
df = pd.read_csv("/kaggle/working/train_bi_2.csv")
df.head()

,query,answer,cid,qid
0,Người học ngành quản_lý khai_thác công_trình t...,"Khả_năng học_tập , nâng cao_trình_độ - Khối_lư...",62492,161615
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,151154,80037
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"Điều 20 . Ưu_đãi về thời_gian miễn thuế , gi...",75071,124074
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"Điều_kiện được hưởngCán bộ quản_lý , giáo_viên...",225897,146841
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"Nguyên_tắc áp dụng 1 . Trường_hợp công_chức ,...",68365,6176
